In [ ]:
import os
import numpy as np
from segment_anything import SamPredictor, sam_model_registry
from PIL import Image
import torch

# Путь к папке с изображениями и разметкой
images_folder = "data_yolo_29_07_v2(copy)/images/train"
labels_folder = "data_yolo_29_07_v2(copy)/labels/train"
output_masks_folder = "masks"

# Создаем папку для сохранения масок, если ее нет
os.makedirs(output_masks_folder, exist_ok=True)

# Путь к уже скачанной модели SAM
sam_checkpoint = "sam_vit_b_01ec64.pth"

# Загружаем модель SAM
model_type = "vit_b"  # Тип модели
sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
predictor = SamPredictor(sam)

# Определяем цвета для каждого класса (6 классов)
# Преобразуем HEX-цвета в RGB
def hex_to_rgb(hex_color):
    hex_color = hex_color.lstrip("#")
    return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))

class_colors = {
    0: hex_to_rgb("#fa3253"),  # bird
    1: hex_to_rgb("#24b353"),  # helicopter
    2: hex_to_rgb("#fafa37"),  # airplane
    3: hex_to_rgb("#733380"),  # drone
    4: hex_to_rgb("#34d1b7"),  # ballon
    5: hex_to_rgb("#66ff66"),  # ballon_c
}

# Проходимся по всем изображениям
for image_name in os.listdir(images_folder):
    if image_name.endswith(('.png', '.jpg', '.jpeg')):
        print(f"Обрабатывается изображение: {image_name}")
        
        # Загружаем изображение
        image_path = os.path.join(images_folder, image_name)
        image = np.array(Image.open(image_path).convert("RGB"))
        predictor.set_image(image)

        # Загружаем соответствующий файл разметки YOLO
        label_path = os.path.join(labels_folder, os.path.splitext(image_name)[0] + ".txt")
        if not os.path.exists(label_path):
            print(f"Файл разметки не найден для изображения {image_name}")
            continue

        # Инициализируем пустую маску RGB
        h, w, _ = image.shape
        combined_mask = np.zeros((h, w, 3), dtype=np.uint8)

        with open(label_path, "r") as f:
            lines = f.readlines()

        # Обрабатываем каждую метку (bounding box) из файла разметки
        for line in lines:
            # Формат строки разметки: class_id center_x center_y width height
            data = line.strip().split()
            class_id, center_x, center_y, width, height = map(float, data)

            # Преобразуем координаты из YOLO формата в пиксели
            x_min = int((center_x - width / 2) * w)
            y_min = int((center_y - height / 2) * h)
            x_max = int((center_x + width / 2) * w)
            y_max = int((center_y + height / 2) * h)

            # Создаем маску с помощью SAM
            box = np.array([x_min, y_min, x_max, y_max])
            masks, _, _ = predictor.predict(box=box, point_coords=None, point_labels=None, multimask_output=False)

            # Преобразуем маску в изображение и объединяем ее с основной маской
            mask_np = masks[0].astype(np.uint8)
            color = class_colors.get(int(class_id), (255, 255, 255))  # Цвет по умолчанию - белый

            # Применяем цвет к маске
            for c in range(3):
                combined_mask[:, :, c] = np.where(mask_np > 0, color[c], combined_mask[:, :, c])

        # Сохраняем объединенную маску как изображение
        combined_mask_image = Image.fromarray(combined_mask)
        combined_mask_filename = f"{os.path.splitext(image_name)[0]}.png"
        combined_mask_image.save(os.path.join(output_masks_folder, combined_mask_filename))

print("Маски успешно созданы и сохранены!")
